In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Proyecto de Detección de Fraude En Transacciones Bancarias
Antonio Montilla

Neoland

Madrid, 15 de Marzo de 2020

## Introducción
La forma en que los consumidores realizamos nuestras compras habituales de bienes y servicios ha sufrido una gran trasnformación en las últimas décadas, caracterizada por el mayor uso de dinero electrónico como medio de pago. La bancarización en los países desarrollados ha alcanzado nuevos niveles de penetración, que, en conjunto con la masificación en el uso del internet, ha propulsado el comercio en linea, o e-commerce.

Con el desarrollo tecnológico, las transacciones que utilizan tarjetas de crédito/débito o, incluso, medios electrónicos de pago a través de dispositivos móviles (como Apple pay o Paypal), son cada vez más frecuentes. En España, por ejemplo, las transacciones de este tipo, es decir aquellas que se efectuan sin efectivo, ya superan el 10% del total de las compras para el año 2018, de acuerdo a estimaciones del Banco Central Europeo. En EEUU, la proporsión es de 75%, de acuerdo a a la Reserva Federal, mientras que en países como Suecia estas transacciones alcanzan niveles superiores al 80% del total, según estimaciones del Riksbank.

Sin embargo, este desarrollo del mercado bancario y del e-commerce ha traido consigo la proliferación de fraudes, ya sea a través del robo de los medios electrónicos de pago o simplemente mediante la usurpación de la identidad. Como referencia, el Banco Central Europeo estima que el fraude significa costes al sector financiero en torno a 1.800 millones de euros en la zona monetaria europea; en España la cifra se ubica en torna a 90 millones de euros. Desarrollar herramientas que permitan minimizar dichos costes resulta, definitivamente, crucial para la sostenibilidad y el desarrollo de estos avances tecnológicos en el consumo privado.

En este contexto, este proyecto intenta desarrollar modelos que permitirían predecir si una transacción es de índole fraudulante o no, a través del análisis y explotación de una base de datos que contiene información de 590.540 transacciones efectuadas en linea a través de la plataforma de Vesta. El proyecto evalúa las variables más relevantes para predecir la verocidad de una transacción, construye modelos y selecciona el que mejor se adapta a los datos.

El uso de este tipo de metodología, si aplicada masivamente en el mundo real, resultaría en una disminución significativa de las pérdidas ocasianadas por el fraude, tanto a instituciones financieras como al usuario final.



## Objetivos
* Explorar la database provista por Vesta, seleccionar las variables más relevantes para la predicción de la columna que comprueba si la transacción es fradulenta o no.
* Diseñar y construir diferentes modelos usando la base de dato, que permita dicha predicción.
* Evaluar la validez de cada modelo y eligir el que mejor se aplica para este ejercicio.
* Ilustrar el aporte al sector bancario de la implementación de este tipo de algoritmos.


## Proceso de captura de datos
Este proyecto se fundamenta en el análisis y explotación de una base de datos provista por Vesta Corporation, disponible en kaggle (https://www.kaggle.com/c/ieee-fraud-detection). Vesta corporation proveé una plataforma para la realización de operaciones financieras en linea. En este challenge, la compañia instruye a los participantes a proponer modelos de detección de transacciones fradulentas en línea.

La base de dato incluye información de 590.540 transacciones efectuadas en linea a travês de la plataforma de Vesta. En concreto, la base de dato incluye 414 columnas por cada observación, detallando información respecto a la transacción misma (e.g. código de producto, fecha, medio de pago) y la identidad del comprador (e.g. hábitos de consumo, ubicación, dispositivo utilizado). Por tema de seguridad de datos, las columnas referentes a la identidiad son ocultadas.

La variable objetivo es _is Fraud_, que toma valores binarios (0, 1). La base de dato ha sido dividida en un fichero para entrenar el modelo (train) y un fichero para testear (test).

## Cargando las librerias necesarias

In [ ]:
# analisis de datos
import pandas as pd
import numpy as np
import random as rnd
from pandas import read_csv

# visualización
import seaborn as sns
from scipy.stats import norm, skew
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

## scikit modeling libraries
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                             GradientBoostingClassifier, ExtraTreesClassifier,
                             VotingClassifier)

from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict,
                                     StratifiedKFold, learning_curve)

## Predictive modeling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.feature_selection import RFE

#Principal components & otros
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB


## Importando base de datos

In [ ]:
original = '../input/'
url1 = "/kaggle/input/ieee-fraud-detection/train_identity.csv"
url2 = "/kaggle/input/ieee-fraud-detection/train_transaction.csv"

train_identity = read_csv(url1)
train_transaction = read_csv(url2)


## Exploración Preliminar de la Base de Datos

In [ ]:
#exploración inicial del fichero transaction
print('El tamaño del fichero train_transaction es: ', train_transaction.shape)

El fichero train transaction contiene 394 columnas, incluyendo transaction ID como identificador de transacción, y 590.540 observaciones.

In [ ]:
train_transaction.columns.values # Nombre de las variables

In [ ]:
train_transaction.info()

Debido al gran número de observaciones, la memoria resulta insuficiente para explorar todos los datos en su conjunto.
Exploraré el fichero identity para poder tomar una decisión del manejo completo de la base

In [ ]:
#exploración inicial del fichero identity
print('El tamaño del fichero train_identity es: ', train_identity.shape)
train_identity.columns.values # Nombre de las variables

El fichero train identity contiene 41 columnas, incluyendo transaction ID como identificador de transacción, y 144.233 observaciones.

Debido a que esta base de dato contiene menos observaciones, procederé a unir los dos dataframes (usando la columna _transaction ID_) y posteriormente eliminaré las observaciones con datos nulos.

In [ ]:
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [ ]:
#Por ahorrar memoria, ahora proceso a eliminar las base de datos _train transaction_ y _train identity_
del train_identity, train_transaction

In [ ]:
#exploración inicial del fichero con todos los datos _train_
print('El tamaño del fichero train es: ', train.shape)

En su conjunto, la base de dato contiene 590.540 observaciones y 434 columnas.

In [ ]:
train.columns.values # Nombre de las variables

Las variables se encuentran agrupadas en función al siguiente criterio:
* _TransactionID_, que representa el identificador de cada transacción.
* _isFraud_, que es la variable objetivo. Toma valores 1 y 0, dependiendo si la transacción es fraude o no.
* _TransactionDT_, indica tiempo desde transacción (en segundos).
* _TransactionAmt_, indica al monto de la transacción (es USD).
* _ProductCD_, el código de producto de la transacción. Variable categórica.
* _card_ (01-06), información sobre la tarjeta de pago, como tipo, categoría, banco, país, entre otros. Variables categóricas.
* _addr_ (01-02), dirección de cliente y de vendedor. Variables categóricas.
* _dist_ (01-02), se refiere a la distancia entre la ubicación del cliente y el vendedor.
* _Pemaildomain_, el dominio del email del comprador. Variable categórica.
* _Remaildomain_, el dominio del email del vendedor. Variable categórica.
* _C_ (01-14), columnas ocultas e incriptadas. Se refieren a variables de conteo, como el número de direcciones asociadas a la tarjeta, números de teléfono, direcciones de email, entre otros, tanto para el comprador como para el vendedor.
* _D_ (01-15), variables del tiempo, como tiempo transacurrido desde última transacción, entre otros.
* _M_ (01-09), variables que indican si hay un match entre la información de compra. Variables categóricas.
* _V_ (01-339), variables provistas por Vesta referente a la transacción, como clasificación, conteo, entre otros.
* _id_ (01-138), variables ocultas e incriptadas. Se refieren a datos de identidad, que por razones de protección no pueden ser reveladas. Datos personales del vendedor y comprador, datos de la conección o equipo (IP, ISP, proxy). 
* _devicetype_, tipo de dispositivo usado por el comprador. Variable categórica.
* _deviceinfo_, información del dispositivo usado por el comprador. Variable categórica.

In [ ]:
#Explorando los datos vacíos para posible eliminación
train.isnull().sum()

In [ ]:
total = train.isnull().sum().sort_values(ascending = False) #sumando valores nulos por columna
porcentaje = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending = False) #y % del total
missing_train_data  = pd.concat([total, porcentaje], axis=1, keys=['Total', 'Porcentaje']) #df para explorar
missing_train_data.head(50)

In [ ]:
missing_train_data.head(235)

* La exploración inicial sugiere que ciertas columnas contienen una gran cantidad de datos vacíos, que pueden llegar a representar casi la totalidad de los datos, es decir 99% del total para ciertas variables de identidad.

* En total, 232 columnas contienen más del 40% de las observaciones vacías; las 202 columnas restantes contienen más del 70% de los datos cargados, es decir como no nulos.

* Como primer paso, procedo a eliminar las 232 columnas con más del 40% de valores nulos.

In [ ]:
#creo nuevo df con las columnas con más del 70% de las observaciones
Train_new = train.drop(train.loc[:,list((100*(train.isnull().sum()/len(train.index))>30))].columns, 1)
#compruebo presencia de NAs
total1 = Train_new.isnull().sum().sort_values(ascending = False) #sumando valores nulos por columna
porcentaje1 = (Train_new.isnull().sum()/Train_new.isnull().count()*100).sort_values(ascending = False) #y % del total
missing_train_data1  = pd.concat([total1, porcentaje1], axis=1, keys=['Total', 'Porcentaje']) #df para explorar
missing_train_data1.head(50)

In [ ]:
print('El tamaño del fichero train es: ', Train_new.shape)
Train_new.columns.values # Nombre de las variables

Como siguiente caso procedo a eliminar las observaciones nulas, incluyendo tanto valores vacios como NAs, lo que me permitirá reducir el tamaño de la dataframe

In [ ]:
#creo una nueva version del dataset sin valores nulos
Train_new2 = Train_new.dropna()
print('El tamaño del fichero train es: ', Train_new2.shape)

In [ ]:
#compruebo presencia de valores missing y nulos
total1 = Train_new2.isna().sum().sort_values(ascending = False) #sumando valores nulos por columna
porcentaje1 = (Train_new2.isna().sum()/Train_new2.isna().count()*100).sort_values(ascending = False) #y % del total
missing_train_data1  = pd.concat([total1, porcentaje1], axis=1, keys=['Total', 'Porcentaje']) #df para explorar
missing_train_data1.head(50)

Se confirma que no hay observaciones nulos o vacías.

De esta manera, el dataframe se ha reducido a 202 columnas y 328.198 observaciones. Como siguiente paso, evaluaré la presencia de valores vacíos con posibles caracteres especiales, como "?" o "missing".

In [ ]:
interrogante = Train_new2.apply(lambda x: True if "?" in list(x) else False, axis=1)
numOfRows = len(interrogante[interrogante == True].index)
 
print('El número de observaciones con caracter ? es ', numOfRows)

In [ ]:
missing = Train_new2.apply(lambda x: True if "missing" in list(x) else False, axis=1)
numOfRows = len(interrogante[interrogante == True].index)
 
print('El número de observaciones con caracter missing es ', numOfRows)

Se concluye que la dataframe _Train new2_ no contiene valores vacíos, nulos, o con ciertos caracteres especiales.
Ahora procederé a la realización del Exploratory Data Analysis (EDA).

## Exploratory Data Analysis

En esta sección se explorará con más detalle la base de datos, en particular el comportamiento de la variable objetivo _Is Fraud_ y su relación con las demás columnas.

Debido al tamaño de la base de dato, procederé a extraer una muestra del 15% (50.000 datos) del total para facilitar la exploración de los datos.

In [ ]:
# Extrayendo Muestra
Train_sample = Train_new2.sample(frac =.1524, random_state = 2)
print('El tamaño de la muestra de train es: ', Train_sample.shape)

In [ ]:
Train_sample.head()

### Ahora se procederá a la exploración de la variable objetivo _isFraud_

In [ ]:
print("La variable objetivo _Is Fraud_ tiene {0} obervaciones y {1} son valores únicos.".format(Train_sample['isFraud'].count(),Train_sample['isFraud'].nunique()))

In [ ]:
print(Train_sample['isFraud'].describe())

In [ ]:
Train_sample.groupby('isFraud') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribución de IsFraud',
          figsize=(15, 3))
plt.show()

In [ ]:
print(Train_sample['isFraud'].value_counts())

* La variable objetivo toma valores 0 (no fraude) y 1 (fraude).
* Sólo 991 de las observaciones son fraude, es decir 2% del total. En principio, la poca proporsión de observaciones con fraude relativo a los no fraude pudiera incidir en la calidez de los estimadores en el modelado, en sentido que se puede presentar sesgos de estimación.

Ahora se evaluará la relación con cada subset de variables explicativas. 

### TransactionDT
* Columna numérica que indica tiempo desde transacción (en segundos)

In [ ]:
#TransactionDT
print(Train_sample['TransactionDT'].describe())

In [ ]:
#Histograma de columna en log, para suavizar escala
Train_sample['TransactionDT'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          figsize=(15, 5),
          title='Distribución del Log de TransactionDT')
plt.show()

* Variable numérica que mide el tiempo desde transacción.
* Se distribuye con pendiente positiva: los datos se concentran en los valores más altos, es decir, que la moda es que el tiempo transcurrido desde la transacción sea lo más largo.

In [ ]:
#relación con columna objetivo
Train_sample[['TransactionDT', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

In [ ]:
print('La media de TransactionDT con IsFraud igual a 1 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 1]['TransactionDT'].mean()))
print('La media de TransactionDT con IsFraud igual a 0 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 0]['TransactionDT'].mean()))

* En general se puede decir que las trasacciones con fraude se tendieron a producir con menos tiempo desde la recogida de los datos. 
* Sin embargo, dado que la unidad de tiempo es segundos, no se observa una gran diferencia en la media de tiempo transcurrido entre transacciones con fraude o no.
* A priori, la observación sugiere que esta columna no tendrá un nivel explicativo elevado en _IsFraud_.

### TransactionAmt 

* Variable numérica que indica al monto de la transacción, en USD.

In [ ]:
#TransactionAmt
print(Train_sample['TransactionAmt'].describe())

In [ ]:
#Histograma de columna en log, para suavizar escala
Train_sample['TransactionAmt'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          figsize=(15, 5),
          title='Distribución del Log de TransactionAmt')
plt.show()

* En promedio, el monto de una transacción gira en torno a 160 USD; el monto mínimo es de 3,5 USD mientras el máximo es de 4.843 USD
* El log de la serie muestra una distribución que se asemeja a la normal

In [ ]:
#relación con columna objetivo
Train_sample[['TransactionAmt', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

In [ ]:
print('La media de TransactionAmt con IsFraud igual a 1 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 1]['TransactionAmt'].mean()))
print('La media de TransactionAmt con IsFraud igual a 0 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 0]['TransactionAmt'].mean()))

In [ ]:
#Histogramas de TransactionAmt vs. IsFraud
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 6))
Train_sample.loc[Train_sample['isFraud'] == 1] \
    ['TransactionAmt'].apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='Log TransactionAmt vs IsFraud = 1',
          xlim=(-3, 10),
         ax= ax1)
Train_sample.loc[Train_sample['isFraud'] == 0] \
    ['TransactionAmt'].apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='Log TransactionAmt vs IsFraud = 0',
          xlim=(-3, 10),
         ax=ax2)
Train_sample.loc[Train_sample['isFraud'] == 1] \
    ['TransactionAmt'] \
    .plot(kind='hist',
          bins=100,
          title='TransactionAmt vs IsFraud = 1',
         ax= ax3)
Train_sample.loc[Train_sample['isFraud'] == 0] \
    ['TransactionAmt'] \
    .plot(kind='hist',
          bins=100,
          title='TransactionAmt vs IsFraud = 0',
         ax=ax4)
plt.show()

* Las transacciones fradulentas, en promedio, tienden a ser por montos superiores (en torno al 50%) que las transacciones no fradulentas, como en principio se esperaría a priori.
* Esta columna será relevante en predecir si una transacción es fraudulenta.

### ProductCD 

* Variable categórica que indica el código de producto de la transacción.

In [ ]:
print(Train_sample['ProductCD'].value_counts())

In [ ]:
Train_sample.groupby('ProductCD') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribución de ProductCD',
          figsize=(15, 3))
plt.show()

* La exploración muestra que la variable _ProductCD_ toma un único valor (W), lo que resulta inusual.
* Se debe contrastar con la distribución de la columna en la dataframe poblacional para verificar que no sea un problema de muestreo.

In [ ]:
print(Train_new2['ProductCD'].value_counts())

* En la base completa, la columna también toma un valor único.
* Debido a la no variabilidad, procedo a eliminar la columna de mi dataframe, tanto el de muestra como el poblacional.

In [ ]:
#Eliminando la columna _ProductCD_ de las base de datos
Train_sample = Train_sample.drop(['ProductCD'], axis=1)
Train_new2 = Train_new2.drop(['ProductCD'], axis=1)

### Card1 - Card6
* Conjunto de columnas categóricas que denotan información sobre la tarjeta de pago, como tipo, categoría, banco, país, entre otros.

In [ ]:
#creando vector con nombre de las columnas para facilitar exploración
card_cols = ["card1", "card2", "card3", "card4", "card5", "card6"]
Train_sample[card_cols].head(50)

In [ ]:
Train_sample[card_cols].tail(50)

* _Card4_ es categórica e indica el tipo de la tarjeta usada en la operación (visa, mastercard). 
* _Card6_ es categórica e indica la categoría de la tarjeta usada (débito, crédito).
* _Card1, Card2, Card3 y Card5, son numéricas indicando categoría de clasificación de la tarjeta, como código país, banco, antiguedad. El significado no es revelado.

In [ ]:
#Distribución de _Card1_
Train_sample['card1'] \
    .plot(kind='hist',
          bins=100,
          figsize=(7.5, 2.5),
          title='Distribución de card1')
plt.show()

In [ ]:
#Distribución de _Card1_ relativo a IsFraud
Train_sample[['card1', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

* No hay clara distinción de esta clasificación de la tarjeta relativo a la variable objetivo.
* Este comportamiento aplica también a _Card1, Card2 y Card5_ (abajo los gráficos y tablas).

In [ ]:
#Distribución de _Card2_
Train_sample['card2'] \
    .plot(kind='hist',
          bins=100,
          figsize=(7.5, 2.5),
          title='Distribución de card2')
plt.show()

In [ ]:
#Distribución de _Card2_ relativo a IsFraud
Train_sample[['card2', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

In [ ]:
#Distribución de _Card5_
Train_sample['card5'] \
    .plot(kind='hist',
          bins=100,
          figsize=(7.5, 2.5),
          title='Distribución de card5')
plt.show()

In [ ]:
#Distribución de _Card5_ relativo a IsFraud
Train_sample[['card5', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

In [ ]:
#Distribución de _Card3_
Train_sample['card3'] \
    .plot(kind='hist',
          bins=100,
          figsize=(7.5, 2.5),
          title='Distribución de card3')
plt.show()

* La varible toma un único valor (150), que probablemente se refiera al código país de la tarjeta (EEUU).
* Evaluo la distribución en la data poblacional para contrastar. 

In [ ]:
#Valores únicos de _Card3_ en la base poblacional 
print(Train_new2['card3'].value_counts())

* Aunque si muestra multiples valores únicos, estos son insignificantes en términos del tamaño de la muestra: la segunda categoría más repetida incluye menos del 0,1% de todos los datos.
* Debido a esto, procedo a eliminar la variable de las bases de datos, tanto muestral como poblacional.

In [ ]:
#Eliminando la columna _card3_ de las base de datos
Train_sample = Train_sample.drop(['card3'], axis=1)
Train_new2 = Train_new2.drop(['card3'], axis=1)

In [ ]:
#Distribución de variable categórica _Card4_ y _Card6_ referente a tipo de tarjeta
#_Card4_
print(Train_sample['card4'].value_counts())

In [ ]:
Train_sample.groupby('card4') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribución de ProductCD',
          figsize=(15, 3))
plt.show()

In [ ]:
#_Card6_
print(Train_sample['card6'].value_counts())

In [ ]:
Train_sample.groupby('card6') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribución de ProductCD',
          figsize=(15, 3))
plt.show()

* Las transacciones son predominadamente efectuadas a través de tarjetas de débito visa.
* Mastercard es la segunda opción de tipo tarjeta
* Crédito es la segunda opción en términos de categoria

In [ ]:
#Ahora evaluamos la relación de estas con la variable objetivo _IsFraud_
Train_sample_fr1 = Train_sample.loc[Train_sample['isFraud'] == 1]
Train_sample_fr0 = Train_sample.loc[Train_sample['isFraud'] == 0]
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 8))
Train_sample_fr1.groupby('card4')['card4'].count().plot(kind='barh', ax=ax1, title='card4 con IsFraud = 1')
Train_sample_fr0.groupby('card4')['card4'].count().plot(kind='barh', ax=ax2, title='card4 con IsFraud = 0')
Train_sample_fr1.groupby('card6')['card6'].count().plot(kind='barh', ax=ax3, title='card6 con IsFraud = 1')
Train_sample_fr0.groupby('card6')['card6'].count().plot(kind='barh', ax=ax4, title='card6 con IsFraud = 0')
plt.show()

* En términos de tipo de tarjeta, no se observa una gran distición entre operaciones de fraude y no fraude.
* Esto también es cierto para la categoría, si bien, al margen, las tarjetas de crédito tienden a tener un mayor peso relativo en las transacciones de fraude.
* Ahora se realizará la factorización de las columnas, tanto en la muestra como en la población.

In [ ]:
#Card_4
Train_sample['visa'] = np.where(Train_sample['card4'] == 'visa', 1, 0)
print(Train_sample['visa'].value_counts())
Train_sample['mastercard'] = np.where(Train_sample['card4'] == 'mastercard', 1, 0)
print(Train_sample['mastercard'].value_counts())
Train_new2['visa'] = np.where(Train_new2['card4'] == 'visa', 1, 0)
Train_new2['mastercard'] = np.where(Train_new2['card4'] == 'mastercard', 1, 0)

#Card_6
Train_sample['debit'] = np.where(Train_sample['card6'] == 'debit', 1, 0)
print(Train_sample['debit'].value_counts())
Train_sample['credit'] = np.where(Train_sample['card6'] == 'credit', 1, 0)
print(Train_sample['credit'].value_counts())
Train_new2['debit'] = np.where(Train_new2['card6'] == 'debit', 1, 0)
Train_new2['credit'] = np.where(Train_new2['card6'] == 'credit', 1, 0)

#Elimando ambas columnas de las dataframes muestral y poblacional
Train_sample = Train_sample.drop(['card4'], axis=1)
Train_new2 = Train_new2.drop(['card4'], axis=1)
Train_sample = Train_sample.drop(['card6'], axis=1)
Train_new2 = Train_new2.drop(['card6'], axis=1)


### Addr 01-02 
* Variables que indican dirección de comprador y de vendedor. No se revela información adicional.

In [ ]:
#Distribución de _Addr1_
Train_sample['addr1'] \
    .plot(kind='hist',
          bins=100,
          figsize=(7.5, 2.5),
          title='Distribución de addr1')
plt.show()

In [ ]:
#Distribución de _Addr1_ relativo a IsFraud
Train_sample[['addr1', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

* La columna Addr1 se distribuye de manera relativamente uniforme en los datos.
* No se observa, a priori, distinción significativa en esta columna respecto a si hay fraude o no.

In [ ]:
#Distribución de _Addr2_
Train_sample['addr2'] \
    .plot(kind='hist',
          bins=100,
          figsize=(7.5, 2.5),
          title='Distribución de addr2')
plt.show()

* La columna _addr2_ toma un valor unico en la muestra.
* Evaluaré su distribución en la data poblacional.

In [ ]:
#Valores únicos de _addr2_ en la base poblacional 
print(Train_new2['addr2'].value_counts())

* Debido a la no variabilidad en la columna, procedo a eliminar de las bases de datos

In [ ]:
#Eliminando la columna _addr2_ de las base de datos
Train_sample = Train_sample.drop(['addr2'], axis=1)
Train_new2 = Train_new2.drop(['addr2'], axis=1)

### _Pemaildomain_
* el dominio del email del comprador. Variable categórica.

In [ ]:
print(Train_sample['P_emaildomain'].value_counts())

In [ ]:
Train_sample.groupby('P_emaildomain') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribución de P_emaildomain',
          figsize=(15, 15))
plt.show()

In [ ]:
#Ahora evaluamos la relación de estas con la variable objetivo _IsFraud_
#fraude
Train_sample_fr1 = Train_sample.loc[Train_sample['isFraud'] == 1]
print(Train_sample_fr1['P_emaildomain'].value_counts())

In [ ]:
#No fraude
Train_sample_fr0 = Train_sample.loc[Train_sample['isFraud'] == 0]
print(Train_sample_fr0['P_emaildomain'].value_counts())

* Para todas las transacciones, gmail.com es el dominio más común del email del comprador, seguido de yahoo.com y aol.com
* No se observa distinción entre el tipo de transacción, por lo tanto eliminaré la columna tanto de la muestra como de la población

In [ ]:
#Eliminando la columna _P_emaildomain_ de las base de datos
Train_sample = Train_sample.drop(['P_emaildomain'], axis=1)
Train_new2 = Train_new2.drop(['P_emaildomain'], axis=1)

### _C 01-14_ 
* Columnas ocultas e incriptadas. 
* Se refieren a variables de conteo, como el número de direcciones asociadas a la tarjeta, números de teléfono, direcciones de email, entre otros.
* Datos tanto para el comprador como para el vendedor.

In [ ]:
#Creando una dataframes con el conjunto de columnas _C_ para su exploración
c_cols = [c for c in Train_sample if c[0] == 'C']
c_df = Train_sample[c_cols]
c_df.head(50)
c_df.shape

* Debido a la poca información disponible de las 14 columnas de conteo C, procedo a realizar un análisis de componentes principales y extraer nuevas columnas que sintetizen la información.


In [ ]:
#PCA en columnas _C 01-14_

#1) Normalizar ó Estandalizar los datos
scaler=StandardScaler()#instantiate
scaler.fit(c_df) # calcula la media y estandar para cada dimension
X_scaled=scaler.transform(c_df)# transforma los datos a su nueva escala

#2) Aplicando PCA
pca=PCA(n_components=14)
pca.fit(X_scaled) # buscar los componentes principales
X_pca=pca.transform(X_scaled) 
#revisemos la forma del array
print("shape of X_pca", X_pca.shape)

#3) Comprobando variabilidad del PCA
expl = pca.explained_variance_ratio_
print(expl)
print('suma:',sum(expl[0:2]))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

* El análisis de componente principales permite explicar el 97% de la variabilidad de los datos con tan solo 2 nuevas columnas (i.e. componentes).
* Procedo a incluir estas dos componentes en la base de datos _Train sample_ y eliminar las columnas _C 01-14_.
* Posteriormente replicaré este proceso en la base poblacional.

In [ ]:
#Incluyendo las dos nuevas componentes en columnas _C PCA1_ y _C PCA2_
Train_sample['C_PCA1'] = X_pca[:,0]
Train_sample['C_PCA2'] = X_pca[:,1]

#Elimando las columnas C
Train_sample = Train_sample.drop([c for c in c_cols], axis=1)

Train_sample.shape

In [ ]:
#Replicando pasos anteriores en la base poblacional

#PCA en columnas _C 01-14_ en df _Train_new2_

c_df = Train_new2[c_cols]

#1) Normalizar ó Estandalizar los datos
scalerp=StandardScaler()#instantiate
scalerp.fit(c_df) # calcula la media y estandar para cada dimension
X_scaled=scalerp.transform(c_df)# transforma los datos a su nueva escala

#2) Aplicando PCA
pca=PCA(n_components=14)
pca.fit(X_scaled) # buscar los componentes principales
X_pca=pca.transform(X_scaled) 
#revisemos la forma del array
print("shape of X_pca", X_pca.shape)

#3) Comprobando variabilidad del PCA
expl = pca.explained_variance_ratio_
print(expl)
print('suma:',sum(expl[0:2]))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
#Incluyendo las dos nuevas componentes en columnas _C PCA1_ y _C PCA2_ en df _Train_new2_
Train_new2['C_PCA1'] = X_pca[:,0]
Train_new2['C_PCA2'] = X_pca[:,1]

#Elimando las columnas C
Train_new2 = Train_new2.drop([c for c in c_cols], axis=1)

Train_new2.shape

### _V 01-339_
* Conjunto de 169 variables provistas por Vesta referente a la transacción, como clasificación, conteo, entre otros.
* Son numéricas, sin embargo su significando ha sido oculto.
* Procedo a replicar los pasos de análisis de componente principales, así como para las variables _C_

In [ ]:
#Creando una dataframes con el conjunto de columnas _V_ para su exploración
v_cols = [v for v in Train_sample if v[0] == 'V']
v_df = Train_sample[v_cols]
v_df.head(50)
v_df.shape

In [ ]:
#PCA en columnas _V 01-339_

#1) Normalizar ó Estandalizar los datos
scaler=StandardScaler()#instantiate
scaler.fit(v_df) # calcula la media y estandar para cada dimension
X_scaled=scaler.transform(v_df)# transforma los datos a su nueva escala

#2) Aplicando PCA
pca=PCA(n_components=20)
pca.fit(X_scaled) # buscar los componentes principales
X_pca=pca.transform(X_scaled) 
#revisemos la forma del array
print("shape of X_pca", X_pca.shape)

#3) Comprobando variabilidad del PCA
expl = pca.explained_variance_ratio_
print(expl)
print('suma:',sum(expl[0:20]))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

* El análisis de componente principales permite explicar el 74% de la variabilidad de los datos con 20 nuevas columnas (i.e. componentes).
* Procedo a incluir estas veinte componentes en la base de datos _Train sample_ y eliminar las columnas _V 01-33_.
* Posteriormente replicaré este proceso en la base poblacional.

In [ ]:
#Incluyendo las 20 nuevas componentes en columnas _V PCA1_ al _V PCA20_ en df _Train_new2_
Train_sample['V_PCA1'] = X_pca[:,0]
Train_sample['V_PCA2'] = X_pca[:,1]
Train_sample['V_PCA3'] = X_pca[:,2]
Train_sample['V_PCA4'] = X_pca[:,3]
Train_sample['V_PCA5'] = X_pca[:,4]
Train_sample['V_PCA6'] = X_pca[:,5]
Train_sample['V_PCA7'] = X_pca[:,6]
Train_sample['V_PCA8'] = X_pca[:,7]
Train_sample['V_PCA9'] = X_pca[:,8]
Train_sample['V_PCA10'] = X_pca[:,9]
Train_sample['V_PCA11'] = X_pca[:,10]
Train_sample['V_PCA12'] = X_pca[:,11]
Train_sample['V_PCA13'] = X_pca[:,12]
Train_sample['V_PCA14'] = X_pca[:,13]
Train_sample['V_PCA15'] = X_pca[:,14]
Train_sample['V_PCA16'] = X_pca[:,15]
Train_sample['V_PCA17'] = X_pca[:,16]
Train_sample['V_PCA18'] = X_pca[:,17]
Train_sample['V_PCA19'] = X_pca[:,18]
Train_sample['V_PCA20'] = X_pca[:,19]

#Elimando las columnas V
Train_sample = Train_sample.drop([v for v in v_cols], axis=1)

Train_sample.shape

In [ ]:
#Replicando pasos anteriores en la base poblacional

#PCA en columnas _V 01-_339 en df _Train_new2_

v_df = Train_new2[v_cols]

#1) Normalizar ó Estandalizar los datos
scalerp=StandardScaler()#instantiate
scalerp.fit(v_df) # calcula la media y estandar para cada dimension
X_scaled=scalerp.transform(v_df)# transforma los datos a su nueva escala

#2) Aplicando PCA
pca=PCA(n_components=20)
pca.fit(X_scaled) # buscar los componentes principales
X_pca=pca.transform(X_scaled) 
#revisemos la forma del array
print("shape of X_pca", X_pca.shape)

#3) Comprobando variabilidad del PCA
expl = pca.explained_variance_ratio_
print(expl)
print('suma:',sum(expl[0:20]))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
#Incluyendo las 20 nuevas componentes en columnas _V PCA1_ al _V PCA20_ en df _Train_new2_
Train_new2['V_PCA1'] = X_pca[:,0]
Train_new2['V_PCA2'] = X_pca[:,1]
Train_new2['V_PCA3'] = X_pca[:,2]
Train_new2['V_PCA4'] = X_pca[:,3]
Train_new2['V_PCA5'] = X_pca[:,4]
Train_new2['V_PCA6'] = X_pca[:,5]
Train_new2['V_PCA7'] = X_pca[:,6]
Train_new2['V_PCA8'] = X_pca[:,7]
Train_new2['V_PCA9'] = X_pca[:,8]
Train_new2['V_PCA10'] = X_pca[:,9]
Train_new2['V_PCA11'] = X_pca[:,10]
Train_new2['V_PCA12'] = X_pca[:,11]
Train_new2['V_PCA13'] = X_pca[:,12]
Train_new2['V_PCA14'] = X_pca[:,13]
Train_new2['V_PCA15'] = X_pca[:,14]
Train_new2['V_PCA16'] = X_pca[:,15]
Train_new2['V_PCA17'] = X_pca[:,16]
Train_new2['V_PCA18'] = X_pca[:,17]
Train_new2['V_PCA19'] = X_pca[:,18]
Train_new2['V_PCA20'] = X_pca[:,19]

#Elimando las columnas V
Train_new2 = Train_new2.drop([v for v in v_cols], axis=1)

Train_new2.shape


### _D1_ , _D4_ , _D10_ , _D15_
* Variables numéricas que denotan tiempo, como tiempo transacurrido desde última transacción, entre otros.
* Su significado real es oculto.
* Exploraré indivualmente su comportamiento así como su relación con la variable objetivo _IsFraud_

In [ ]:
#D1
#TransactionDT
print(Train_sample['D1'].describe())

In [ ]:
#Histograma de columna D1
Train_sample['D1'] \
    .plot(kind='hist',
          bins=100,
          figsize=(15, 5),
          title='Distribución de D1 TransactionDT')
plt.show()

In [ ]:
#relación con columna objetivo
Train_sample[['D1', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

In [ ]:
print('La media de D1 con IsFraud igual a 1 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 1]['D1'].mean()))
print('La media de D1 con IsFraud igual a 0 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 0]['D1'].mean()))

* Se observa una diferencia relevante en esta columnas entre transacciones fradulentas y no: para observaciones con fraude, D1 tiende a ser casi la mitad que cuando se trata de no fraude.

In [ ]:
#D4
#TransactionDT
print(Train_sample['D4'].describe())

In [ ]:
#Histograma de columna D4
Train_sample['D4'] \
    .plot(kind='hist',
          bins=100,
          figsize=(15, 5),
          title='Distribución de D4')
plt.show()

In [ ]:
#relación con columna objetivo
Train_sample[['D4', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

In [ ]:
print('La media de D4 con IsFraud igual a 1 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 1]['D4'].mean()))
print('La media de D4 con IsFraud igual a 0 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 0]['D4'].mean()))

* Al igual que con D1, esta columna muestra un comportamiento diferenciado respecto a los valores de la variable objetivo: tiende a ser 50% superior en transacciones no fradulentas.

In [ ]:
#D10
#TransactionDT
print(Train_sample['D10'].describe())

In [ ]:
#Histograma de columna D10
Train_sample['D10'] \
    .plot(kind='hist',
          bins=100,
          figsize=(15, 5),
          title='Distribución de D10')
plt.show()

In [ ]:
#relación con columna objetivo
Train_sample[['D10', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

In [ ]:
print('La media de D10 con IsFraud igual a 1 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 1]['D10'].mean()))
print('La media de D10 con IsFraud igual a 0 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 0]['D10'].mean()))

* Al igual que con D1 y D4, esta columna muestra un comportamiento diferenciado respecto a los valores de la variable objetivo: tiende a ser 50% superior en transacciones no fradulentas.

In [ ]:
#D15
#TransactionDT
print(Train_sample['D15'].describe())

In [ ]:
#Histograma de columna D15
Train_sample['D15'] \
    .plot(kind='hist',
          bins=100,
          figsize=(15, 5),
          title='Distribución de D15')
plt.show()

In [ ]:
#relación con columna objetivo
Train_sample[['D15', 'isFraud']].groupby(['isFraud'], as_index=False).mean()

In [ ]:
print('La media de D15 con IsFraud igual a 1 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 1]['D15'].mean()))
print('La media de D15 con IsFraud igual a 0 es: {:.4f}'.format(Train_sample.loc[Train_sample['isFraud'] == 0]['D15'].mean()))

* Distribución y relación con variable objetivo similar a las otras columnas D

### _M6_
* Variable categórica que denota si existe un match entre la información de compra.

In [ ]:
print(Train_sample['M6'].value_counts())

* _M6_ toma valores 'F' (False) si no hay un match y 'T' (True) si hay un match
* Procederé a sustituir las observaciones con 'T' por 1 y 'F' por 0, de modo de poder ver distribución y relación con columna objetivo

In [ ]:
#Convirtiendo columna _M6_ en binaria, T == 1
Train_sample['M6'] = np.where(Train_sample['M6'] == 'T', 1, 0)
print(Train_sample['M6'].value_counts())

#Replicando para la df poblacional
Train_new2['M6'] = np.where(Train_new2['M6'] == 'T', 1, 0)
print(Train_new2['M6'].value_counts())

In [ ]:
Train_sample.groupby('M6') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribución de M6',
          figsize=(15, 3))
plt.show()

In [ ]:
#relación con columna objetivo
#fraude
Train_sample_fr1 = Train_sample.loc[Train_sample['isFraud'] == 1]
Train_sample_fr1.groupby('M6') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribución de M6 con isFraud = 1',
          figsize=(15, 3))
plt.show()

In [ ]:
#No fraude
Train_sample_fr0 = Train_sample.loc[Train_sample['isFraud'] == 0]
Train_sample_fr0.groupby('M6') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Distribución de M6 con isFraud = 0',
          figsize=(15, 3))
plt.show()

* En línea con lo esperado, en las transacciones fradulentas la proporsión de observaciones en las que no hay un match en la información (respecto al total) tiende a ser notablemente superior que en aquellas transacciones no fradulentas.
* Esta columna debería tener un caracter explicativo en el modelado de _IsFraud_

# EDA Summary
* El análisis exploratorio permitió en balance reducir la base de datos desde 202 columnas a tan solo 39, incluyendo las columnas categóricas factorizadas, lo que ayudará en el proceso de modelado y evaluación.
* Se practicaron eliminación de datos vacios, nulos, transformaciones de las variables, análisis de componentes principales y factorización.
* Se identificó también el riesgo de que la base de dato sea no balanceada: la variable objetivo incluye solo el 2% de las observaciones como fraude. Esto pudiera suponer problemas en el proceso de modelado.
* Como paso adicional, procederé a realizar análisis de feature engeniring antes del modelado.

In [ ]:
#Creo un backup de las bases de datos por resguardo antes de eliminar nuevas columnas.
Train_sample_bk = Train_sample
Train_new2_bk = Train_new2

In [ ]:
Train_new2_bk.info()

### Matriz de correlaciones

In [ ]:
correlation_matrix = Train_sample.corr()
correlation_matrix

plt.figure(figsize=(23.0,23.0))
plt.title('Matriz de Correlación de Pearson')
sns.heatmap(correlation_matrix, annot=True)

* El análisis inicial sugiere que la variable objetivo se correlaciona más con el conjunto de columnas _D_, tipo de tarjeta (débito o crédito), y algunas componentes principales de las columnas _C_ y _V_
* De igual manera, se puede observar que hay ciertas columnas que no parecen estar relacionadas con la variable objetivo y podría ser util extraerlas del database.
* Se observará la matriz de correlación solo para el top 15 de las columnas más correlacionadas. 

In [ ]:
corrmat = Train_sample.corr()
cols_corrmat = corrmat['isFraud'].abs()
cols_corrmat = cols_corrmat.sort_values(ascending=False)
cols_corrmat.head(39)

* A partir de la columna 13, el coeficiente de correlación con _Is Fraud_ cae por debajo del 3%, en sentido directo o inversamente.
* Se tomarán estas primeras 13 columnas para el proceso de modelado.

In [ ]:
#Guardando columnas finales en array para proceso de modelado
cols = cols_corrmat.index
cols = cols[0:14]
cols
df_model = Train_new2[cols]
df_model.shape

Ahora guardaré la base de dato original,muestral y la que se utilizará en el proceso de modelado en CSV indiduales.

In [ ]:
Train_new2.to_csv('Train_new2.csv',index=False)
Train_sample.to_csv('Train_sample.csv',index=False)
df_model.to_csv('df_model.csv',index=False)

### Modelos

Continua en un notebook separado